# Time series prediction with XGBoost

Goal is to build model that will predict utilization of pool of bicycles in major US city for the next day.

## Forecasting scenario 

Forecast each day at 23:30 values for the next day (24 values ahead).

## Data

CSV file contains columns: timestamp, cnt, temp, atemp, hum, windspeed, holiday

Target variable to be predicted: cnt

Data availability:

- Target:  till last hour of prev. day i.e. 23:00 (D-1)
- All predictors - till end of prediction horizon, i.e. till 23:00 next day (D+1)

## Source:

Dataset was taken from Kaggle website. 


In [62]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from xgboost import XGBRegressor

import datetime

Read data, set timestamp as index.

In [63]:
df = pd.read_csv('data/Bike_sharing.csv', sep = ';')

In [64]:
df['Timestamp'] = pd.to_datetime( df['Timestamp'] )

In [65]:
df.set_index('Timestamp', inplace=True)

In [66]:
fig = go.Figure()

fig.add_trace( go.Scatter( x=df.index, y=df['cnt'], mode='lines', name='cnt' ) )

fig.update_layout( width=1000, height=500 )
fig.show()   

Preview and explore

In [67]:
df.describe()

,cnt,temp,atemp,hum,windspeed,holiday
count,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,189.463088,0.496987,0.475775,0.627229,0.190098,0.028770
std,181.387599,0.192556,0.171850,0.192930,0.122340,0.167165
min,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,0.340000,0.333300,0.480000,0.104500,0.000000
50%,142.000000,0.500000,0.484800,0.630000,0.194000,0.000000
75%,281.000000,0.660000,0.621200,0.780000,0.253700,0.000000
max,977.000000,1.000000,1.000000,1.000000,0.850700,1.000000


There are no mising values

In [68]:
df.isnull().sum()

cnt          0
temp         0
atemp        0
hum          0
windspeed    0
holiday      0
dtype: int64

In [69]:
df.head()

,cnt,temp,atemp,hum,windspeed,holiday
Timestamp,,,,,,
2011-01-01 00:00:00,16,0.24,0.2879,0.81,0.0,0
2011-01-01 01:00:00,40,0.22,0.2727,0.80,0.0,0
2011-01-01 02:00:00,32,0.22,0.2727,0.80,0.0,0
2011-01-01 03:00:00,13,0.24,0.2879,0.75,0.0,0
2011-01-01 04:00:00,1,0.24,0.2879,0.75,0.0,0


In [70]:
df.tail()

,cnt,temp,atemp,hum,windspeed,holiday
Timestamp,,,,,,
2012-12-31 19:00:00,119,0.26,0.2576,0.60,0.1642,0
2012-12-31 20:00:00,89,0.26,0.2576,0.60,0.1642,0
2012-12-31 21:00:00,90,0.26,0.2576,0.60,0.1642,0
2012-12-31 22:00:00,61,0.26,0.2727,0.56,0.1343,0
2012-12-31 23:00:00,49,0.26,0.2727,0.65,0.1343,0


Sampling period is on hourly basis, there are some gaps though

In [71]:
( df.index[1:] - df.index[:-1]).unique()

TimedeltaIndex(['0 days 01:00:00', '0 days 02:00:00', '0 days 03:00:00',
                '0 days 13:00:00', '0 days 23:00:00', '0 days 07:00:00',
                '0 days 14:00:00', '1 days 13:00:00'],
               dtype='timedelta64[ns]', name='Timestamp', freq=None)

Spans through 2 years

In [72]:
df.index[-1] - df.index[0]

Timedelta('730 days 23:00:00')

Distribution of target variable

In [73]:
fig = px.histogram(df['cnt'], x='cnt')
fig.show()

Helper functions

In [2]:
# Split data
def splitDataByWindowAtAndNormalize( df, at, windowLength, target ):
    if target is None: return

    at = at.strftime("%Y-%m-%d %H:%M:%S")

    features = [ x for x in df.columns.tolist() if x != target ]
    
    X = df[ features ]
    y = df[ target ]

    X = ( X - X.mean() ) / X.std()

    idxAt = df.index.get_loc(at)

    X_train = X[ : idxAt ]    
    y_train = y[ : idxAt ]
    
    X_test = X.iloc[ idxAt : idxAt + windowLength ]
    y_test = y.iloc[ idxAt : idxAt + windowLength ]
        
    return X_train, y_train, X_test, y_test 


In [3]:
# Evaluate residuals
def buildEvalDf( df ):
    df['err'] = df['y'] - df['y_pred']
    df['errpctabs'] = np.abs( ( df['y'] - df['y_pred'] ) / df['y'] )
    return df

def evaluate( df ):    
    # metrics
    print( 'Accuracy metrics' )
    print( 'MAE:', mean_absolute_error( df['y'].values, df['y_pred'].values) )
    print( 'RMSE:', np.sqrt( mean_squared_error( df['y'].values, df['y_pred'].values) ) )    
    print( 'MAPE (%):', df['errpctabs'].mean()*100 )
    print( 'Min. Error:', df['err'].min() )
    print( 'Max. Error:', df['err'].max() )

    # plot 
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df['y'], mode='lines', name='y'))
    fig.add_trace(go.Scatter(x=df.index, y=df['y_pred'], mode='lines', name='y_pred'))
    fig.update_layout(  title='Actual vs. Predicted', width=1000, height=500  )
    fig.show()   

    # resid. distribution
    fig = px.histogram(df['err'], x='err')
    fig.update_layout(  title='Residuals distribution' )
    fig.show()


In [16]:
TARGET = 'cnt'
PREDICTORS = [ x for x in df.columns.tolist() if x != TARGET ]

Enhance data with simple features

In [17]:
# Weekday
df['weekday'] = df.index.weekday

Prepare data for each prediction point of horizon, i.e. per each hour, to reflect on expected, real, shape of data that will occur in production.

Target values must not leak, i.e. if predicting on TUE at 23:30, the last known actual value is from day before, MON 23:00.
Predictors on the other hand are available up until end of prediction horizon, i.e. on TUE at 23:30, values are available up until WED 23:00.

The look back period will be limited for the last 7 days.

In [18]:
# Separate data by hour
dfh = {}
for h in range(0,24): dfh[ h ] = df[ df.index.hour == h ]

# visor for data
LOOKBACK_START_PREDICTORS = 0
LOOKBACK_END = 24*7

# For each hour (point) in prediction horizon
for h_in_prediction_horizon in range(0,24): 
    # look back to values available for particular point: 
    #    - for target
    for lookback in range( h_in_prediction_horizon + 1, LOOKBACK_END ): 
        dfh[ h_in_prediction_horizon ].loc[ : , TARGET+'_t-'+str( lookback ) ] = df[ TARGET ].shift( lookback )

    #    - and predictors
    for c in PREDICTORS:
        for lookback in range( LOOKBACK_START_PREDICTORS, LOOKBACK_END ):
            dfh[ h_in_prediction_horizon ].loc[ : , c+'_t-'+str( lookback ) ] = df[ c ].shift( lookback )

    # Clean up records without values
    dfh[ h_in_prediction_horizon ].dropna( inplace=True )


In [19]:
dfh[5].head()

,cnt,temp,atemp,hum,windspeed,holiday,weekday,cnt_t-6,cnt_t-7,cnt_t-8,...,holiday_t-158,holiday_t-159,holiday_t-160,holiday_t-161,holiday_t-162,holiday_t-163,holiday_t-164,holiday_t-165,holiday_t-166,holiday_t-167
Timestamp,,,,,,,,,,,,,,,,,,,,,
2011-01-09 05:00:00,1,0.08,0.0909,0.53,0.1940,0,6,22.0,34.0,37.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-10 05:00:00,3,0.10,0.1061,0.54,0.2537,0,0,6.0,15.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-11 05:00:00,6,0.16,0.1818,0.55,0.1343,0,1,38.0,74.0,95.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-12 05:00:00,5,0.14,0.1515,0.86,0.1642,0,2,20.0,32.0,51.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-13 05:00:00,3,0.14,0.1212,0.50,0.2985,0,3,20.0,33.0,57.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
dfh[5].shape[1]

1009

Back-test data on out-of-sample interval

In [44]:
xgbr = XGBRegressor( n_estimators=500, objective ='reg:squarederror', learning_rate = 0.05, max_depth = 3, min_child_weight = 3 )

In [52]:
windowInDays = 7
windowStarts = [ datetime.datetime(2012,6,4) + datetime.timedelta( days=windowInDays) * x for x in range(0,30) ]

In [53]:
windowStarts

[datetime.datetime(2012, 6, 4, 0, 0),
 datetime.datetime(2012, 6, 11, 0, 0),
 datetime.datetime(2012, 6, 18, 0, 0),
 datetime.datetime(2012, 6, 25, 0, 0),
 datetime.datetime(2012, 7, 2, 0, 0),
 datetime.datetime(2012, 7, 9, 0, 0),
 datetime.datetime(2012, 7, 16, 0, 0),
 datetime.datetime(2012, 7, 23, 0, 0),
 datetime.datetime(2012, 7, 30, 0, 0),
 datetime.datetime(2012, 8, 6, 0, 0),
 datetime.datetime(2012, 8, 13, 0, 0),
 datetime.datetime(2012, 8, 20, 0, 0),
 datetime.datetime(2012, 8, 27, 0, 0),
 datetime.datetime(2012, 9, 3, 0, 0),
 datetime.datetime(2012, 9, 10, 0, 0),
 datetime.datetime(2012, 9, 17, 0, 0),
 datetime.datetime(2012, 9, 24, 0, 0),
 datetime.datetime(2012, 10, 1, 0, 0),
 datetime.datetime(2012, 10, 8, 0, 0),
 datetime.datetime(2012, 10, 15, 0, 0),
 datetime.datetime(2012, 10, 22, 0, 0),
 datetime.datetime(2012, 10, 29, 0, 0),
 datetime.datetime(2012, 11, 5, 0, 0),
 datetime.datetime(2012, 11, 12, 0, 0),
 datetime.datetime(2012, 11, 19, 0, 0),
 datetime.datetime(2012, 

Data seems quite unstable, it is expected that if model is built on the first 2/3 of interval and used to predict all timestamps of remainng 1/3, the result will be unsatisfactory. Instead, I will back-test on rolling window principle, simulate weekly model rebuilding for weeks starting with timestamps listed above, so out-of-sample interval of each rolling window will be just 1 week long.

In [54]:
start_time = datetime.datetime.now()  

y, y_pred = pd.DataFrame(), pd.DataFrame()

progressWindow = 0
for ws in windowStarts:  
    # we will create model for each hour of prediction horizon
    for h_in_prediction_horizon in range(0,24):
        cutAt_ = ws + datetime.timedelta( hours = h_in_prediction_horizon)

        if cutAt_ in df.index:
            X_train, y_train, X_test, y_test = splitDataByWindowAtAndNormalize( dfh[ h_in_prediction_horizon ], cutAt_ , windowInDays, TARGET )        

            xgbr.fit(X_train, y_train)
            y_pred_ = xgbr.predict( X_test )
            
            y_pred_df_ = pd.DataFrame( {'y_pred': y_pred_} )
            y_pred_df_.index = y_test.index.tolist()
            y_pred = y_pred.append( y_pred_df_ )
            
            y = y.append( pd.DataFrame( {'y':y_test } ) )

            # print progress
            print('Hour:', h_in_prediction_horizon, '/ Interim time:', datetime.datetime.now() - start_time )
        else:
            print('Skipping',cutAt_)
        
    # print progress
    progressWindow+=1  
    print('-------')
    print('Progress %', progressWindow / len(windowStarts) * 100, '/ Interim time:', datetime.datetime.now() - start_time )

print( 'Total time:', datetime.datetime.now() - start_time )

 Interim time: 0:10:29.247477
Hour: 2 / Interim time: 0:10:31.857634
Hour: 3 / Interim time: 0:10:34.418840
Hour: 4 / Interim time: 0:10:37.073252
Hour: 5 / Interim time: 0:10:39.681363
Hour: 6 / Interim time: 0:10:42.309326
Hour: 7 / Interim time: 0:10:44.933928
Hour: 8 / Interim time: 0:10:47.531746
Hour: 9 / Interim time: 0:10:50.086938
Hour: 10 / Interim time: 0:10:52.734722
Hour: 11 / Interim time: 0:10:55.323312
Hour: 12 / Interim time: 0:10:57.894752
Hour: 13 / Interim time: 0:11:00.453008
Hour: 14 / Interim time: 0:11:03.102441
Hour: 15 / Interim time: 0:11:06.084222
Hour: 16 / Interim time: 0:11:08.659280
Hour: 17 / Interim time: 0:11:11.635590
Hour: 18 / Interim time: 0:11:15.568968
Hour: 19 / Interim time: 0:11:18.477551
Hour: 20 / Interim time: 0:11:21.384538
Hour: 21 / Interim time: 0:11:24.078418
Hour: 22 / Interim time: 0:11:28.303443
Hour: 23 / Interim time: 0:11:30.943037
-------
Progress % 36.666666666666664 / Interim time: 0:11:30.943804
Hour: 0 / Interim time: 0:11:

In [55]:
df_xgbr = y.join(y_pred).sort_index()

Save results to pickle file on hard drive - it was time consuming to produce eval. dataframe, lets save it for later exploration

In [56]:
df_xgbr.to_pickle('df_xgbr.pkl') 

In [57]:
df_xgbr = pd.read_pickle('df_xgbr.pkl')

In [58]:
df_xgbr.head()

,y,y_pred
2012-06-04 00:00:00,49,63.664169
2012-06-04 01:00:00,14,27.527905
2012-06-04 02:00:00,11,14.773041
2012-06-04 03:00:00,5,7.876263
2012-06-04 04:00:00,8,7.924020


Evaluate

In [59]:
evaluate( buildEvalDf( df_xgbr ) )

Accuracy metrics
MAE: 40.84035020636554
RMSE: 65.67981903709389
MAPE (%): 30.34172001708634
Min. Error: -532.1166381835938
Max. Error: 355.0857849121094


Night hours (0-5) seems to have lower accuracy than day hours

In [60]:
x = [ h for h in range(0,24)]
y = [ df_xgbr[ df_xgbr.index.hour==h ]['errpctabs'].median() for h in range(0,24) ]

fig = go.Figure( data=[ go.Bar( x=y, y=x, text=x, textposition='auto', orientation='h' ) ])

fig.update_layout( width=500, height=700 , title='Error by hour of the day')
fig.show()

Weekend days seems to be least accurate

In [61]:
x = [ wd for wd in range(0,7)]
y = [ df_xgbr[ df_xgbr.index.weekday==wd ]['errpctabs'].median() for wd in range(0,7) ]

fig = go.Figure( data=[ go.Bar( x=y, y=x, text=x, textposition='auto', orientation='h' ) ])

fig.update_layout( width=500, height=500 , title='Error by day of week')
fig.show()

Next steps to improve the models:

- try log of target variable
- check on problematic weekdays and night hours